In [1]:
import tensorflow as tf
import keras

Using TensorFlow backend.


In [2]:
from keras.backend import tf as K
K.reset_default_graph()
#K.global_variables_initializer()

In [3]:
import keras

In [4]:
cd G:/RSAVED/SCRIPTS

G:\RSAVED\SCRIPTS


In [5]:
keras.__version__

'2.0.8'

In [6]:
#! pip install flask 
import h5py


In [7]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1123345012067605547
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 5391949824
locality {
  bus_id: 1
}
incarnation: 7407872422734146621
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:02:00.0"
]


In [8]:
from keras.backend import tf as K
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers.normalization import  BatchNormalization
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from sklearn.metrics import log_loss
from keras.applications import VGG16, InceptionV3, MobileNet, ResNet50, VGG19, Xception
#conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(768, 1024, 3))# initialize with your own image format


In [9]:
#conv_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(768, 1024, 3))# initialize with your own image format
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(192, 256, 3))# initialize with your own image format


In [10]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 192, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 192, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 192, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 96, 128, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 96, 128, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 96, 128, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 48, 64, 128)       0         
__________

In [11]:
drop=0.7
from keras import models
from keras import layers
from keras import optimizers
with tf.device('/gpu:0'):
    model = models.Sequential()
    for layer in conv_base.layers[:-5]:
        layer.trainable = False
        model.add(layer)
    #model.add(conv_base)
#with tf.device('/gpu:0'):
    model.add(MaxPooling2D((2,2), strides=(2,2))) 
    model.add(Convolution2D(512, (3, 3), activation='relu')) 
    model.add(ZeroPadding2D((1,1))) 
    model.add(Convolution2D(512, (3, 3), activation='relu')) 
    model.add(ZeroPadding2D((1,1))) 
    model.add(Convolution2D(512, (3, 3), activation='relu')) 
    model.add(ZeroPadding2D((1,1))) 
    model.add(Convolution2D(512, (3, 3), activation='relu')) 
    model.add(MaxPooling2D((2,2), strides=(2,2))) 
    model.add(layers.Flatten())
    model.add(BatchNormalization())
    model.add(layers.Dense(256, activation='relu'))
    model.add(Dropout(drop))
    model.add(BatchNormalization())
    model.add(layers.Dense(64, activation='relu'))
    model.add(Dropout(drop))
    model.add(BatchNormalization())
    model.add(layers.Dense(16, activation='relu'))
    model.add(Dropout(drop))
    model.add(BatchNormalization())
    model.add(layers.Dense(1, activation='sigmoid'))

In [12]:
#for layer in conv_base.layers:
 #   layer.trainable = False
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 192, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 192, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 192, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 96, 128, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 96, 128, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 96, 128, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 48, 64, 128)       0         
__________

In [13]:
import numpy as np
from  skimage import io
import pandas as pd


In [14]:
cd G:\rsaved\ISIC-images\im

G:\rsaved\ISIC-images\im


In [15]:
filename =  'Benign/ISIC_0009872.jpg'
y=filename.split('/')[0]
X=io.imread('Benign/ISIC_0009872.jpg')

In [16]:

# extract features for images and create labels from the names of the folders
import os, sys 
from skimage.transform import rescale, resize, downscale_local_mean

def resizeimage(IMAGES,factor):
    ima=[]
    try:
        image_rescaled = rescale(IMAGES, factor)
        #ima.append(out)

    except Exception as e:
        print(e)
    return(image_rescaled)
#mod = resizeimage(X_train)

def extract(filename =  'Benign/ISIC_0009872.jpg'):
    X=0
    y=0
    try:
        print(filename)
        X=io.imread(filename) # extract pixel features 
        X= resizeimage(X,0.25)
        y=filename.split('/ISIC')[0]
        if y=='Benign': # create label
            y=0
        else:
            y=1
        print(y)
    except:
        print('error' + filename)
    return X,y
q=extract()

Benign/ISIC_0009872.jpg
0


G:\TESTS\envs\kerasbefore\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [17]:
def createdata(): 
    X_train = []
    y_train=[]
    counter = 0
    names = ['Benign','Malignant']
    for folder in names:
        count=0
        print(folder)

        for fil in os.listdir(folder):
            if count <355:

                q=None

                try:
                    q=extract(folder+'/'+fil)
                    #print(q[0][2].shape)
                    if q[0][2].shape == (256,3):
                        X_train.append(q[0])
                        y_train.append(q[1])
                        count +=1
                        counter +=1
                except:
                    pass
            else:
                continue
        print(count,fil)
    return(X_train,y_train)
#X, y = createdata()        

In [21]:
from keras.layers import Lambda, Input
X_train = []
y_train=[]
counter = 0 

def createdata(maxclass): 
    names = ['Benign','Malignant']
    for folder in names:
        count=0
        print(folder)
        
        for fil in os.listdir(folder):
            if count <maxclass:
                count +=1
                q=None
                #print(fil)
                try:
                    q=extract(folder+'/'+fil)
                    print(q[0][2].shape)
                    if q[0][2].shape == (256,3):
                        X_train.append(q[0])
                        y_train.append(q[1])
                        counter +=1
                except:
                    pass
            else:
                continue
    return(X_train,y_train)
#X, y = createdata(1000)     

In [20]:
len(X),len(y)

(1351, 1351)

In [22]:
# convert the X_train list into an array with one additionnal dimension by stacking elements of the list 
c= np.stack([X[i] for i in range(len(X))])

d= np.array([y]).T

In [23]:
d

array([[0],
       [0],
       [0],
       ..., 
       [1],
       [1],
       [1]])

In [24]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(c, d, random_state=3)
#X_train=X_train-np.mean(X_train)/np.std(X_train)

In [25]:
np.sum(y_train)

353

In [27]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.rmsprop(0.0085), metrics=['accuracy'])

In [68]:
#CALLBACKS
from keras.callbacks import *
#This line creates a Callback Tensorboard object, you should capture that object and give it to the fit function of your model.
tense = keras.callbacks.TensorBoard(log_dir='C:/RSAVED/Scripts/Graph', histogram_freq=1, write_graph=True, write_images=False)

class_weight = {0 : 1.,1: 2.87}
# create checkpoints that we can load later to keep the best model 
from keras.callbacks import ModelCheckpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tense.set_model(model)


In [ ]:
#running the model with callbacks
model.fit(X_train[0:1100], y_train[0:1100], nb_epoch=500, batch_size=7, verbose=2, validation_split=0.2, shuffle=True, class_weight = class_weight ,callbacks=[tense,checkpoint])

G:\TESTS\envs\kerasbefore\lib\site-packages\keras\models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 880 samples, validate on 220 samples
Epoch 1/500
Epoch 00000: val_acc improved from -inf to 0.86364, saving model to weights-improvement-00-0.86.hdf5
26s - loss: 0.7153 - acc: 0.7875 - val_loss: 1.8316 - val_acc: 0.8636
Epoch 2/500
Epoch 00001: val_acc did not improve
24s - loss: 0.8089 - acc: 0.7670 - val_loss: 2.5493 - val_acc: 0.5591
Epoch 3/500
Epoch 00002: val_acc did not improve
24s - loss: 0.7976 - acc: 0.7580 - val_loss: 1.5519 - val_acc: 0.8091
Epoch 4/500
Epoch 00003: val_acc did not improve
24s - loss: 0.8603 - acc: 0.7511 - val_loss: 1.2054 - val_acc: 0.7955
Epoch 5/500
Epoch 00004: val_acc did not improve
24s - loss: 0.8090 - acc: 0.7250 - val_loss: 1.9415 - val_acc: 0.8045
Epoch 6/500
Epoch 00005: val_acc improved from 0.86364 to 0.87727, saving model to weights-improvement-05-0.88.hdf5
24s - loss: 0.8308 - acc: 0.7455 - val_loss: 1.2453 - val_acc: 0.8773
Epoch 7/500
Epoch 00006: val_acc did not improve
24s - loss: 0.7479 - acc: 0.7966 - val_loss: 1.8413 - val_ac

Epoch 00067: val_acc did not improve
24s - loss: 0.6916 - acc: 0.8011 - val_loss: 1.4448 - val_acc: 0.8545
Epoch 69/500
Epoch 00068: val_acc did not improve
24s - loss: 0.6316 - acc: 0.8341 - val_loss: 2.3062 - val_acc: 0.8364
Epoch 70/500
Epoch 00069: val_acc did not improve
24s - loss: 0.7010 - acc: 0.8034 - val_loss: 2.2869 - val_acc: 0.8227
Epoch 71/500
Epoch 00070: val_acc did not improve
24s - loss: 0.6850 - acc: 0.8091 - val_loss: 3.5539 - val_acc: 0.8136
Epoch 72/500
Epoch 00071: val_acc did not improve
25s - loss: 0.6992 - acc: 0.7989 - val_loss: 3.2885 - val_acc: 0.7455
Epoch 73/500
Epoch 00072: val_acc did not improve
24s - loss: 0.6937 - acc: 0.8193 - val_loss: 1.3284 - val_acc: 0.7455
Epoch 74/500
Epoch 00073: val_acc did not improve
24s - loss: 0.6254 - acc: 0.8159 - val_loss: 2.0278 - val_acc: 0.7227
Epoch 75/500
Epoch 00074: val_acc did not improve
25s - loss: 0.6009 - acc: 0.8420 - val_loss: 3.4640 - val_acc: 0.7864
Epoch 76/500
Epoch 00075: val_acc did not improve
24s

Epoch 00134: val_acc did not improve
24s - loss: 0.6299 - acc: 0.8216 - val_loss: 2.1862 - val_acc: 0.8136
Epoch 136/500
Epoch 00135: val_acc did not improve
24s - loss: 0.5805 - acc: 0.8500 - val_loss: 3.0028 - val_acc: 0.7909
Epoch 137/500
Epoch 00136: val_acc did not improve
24s - loss: 0.5852 - acc: 0.8523 - val_loss: 2.2759 - val_acc: 0.8273
Epoch 138/500
Epoch 00137: val_acc did not improve
24s - loss: 0.5945 - acc: 0.8591 - val_loss: 2.4857 - val_acc: 0.7955
Epoch 139/500
Epoch 00138: val_acc did not improve
24s - loss: 0.6378 - acc: 0.8193 - val_loss: 1.6099 - val_acc: 0.8045
Epoch 140/500
Epoch 00139: val_acc did not improve
24s - loss: 0.6342 - acc: 0.8114 - val_loss: 1.6384 - val_acc: 0.8273
Epoch 141/500
Epoch 00140: val_acc did not improve
24s - loss: 0.6575 - acc: 0.8307 - val_loss: 1.6868 - val_acc: 0.7182
Epoch 142/500
Epoch 00141: val_acc did not improve
24s - loss: 0.6017 - acc: 0.8580 - val_loss: 1.7957 - val_acc: 0.7864
Epoch 143/500
Epoch 00142: val_acc did not imp

Epoch 00202: val_acc did not improve
25s - loss: 0.6404 - acc: 0.8523 - val_loss: 1.2807 - val_acc: 0.8364
Epoch 204/500
Epoch 00203: val_acc did not improve
24s - loss: 0.6342 - acc: 0.8330 - val_loss: 1.0719 - val_acc: 0.8000
Epoch 205/500
Epoch 00204: val_acc did not improve
25s - loss: 0.5490 - acc: 0.8500 - val_loss: 1.2687 - val_acc: 0.8682
Epoch 206/500
Epoch 00205: val_acc did not improve
24s - loss: 0.6231 - acc: 0.8341 - val_loss: 0.9193 - val_acc: 0.8500
Epoch 207/500
Epoch 00206: val_acc did not improve
24s - loss: 0.4949 - acc: 0.8614 - val_loss: 1.4101 - val_acc: 0.8545
Epoch 208/500
Epoch 00207: val_acc did not improve
25s - loss: 0.6803 - acc: 0.8114 - val_loss: 1.2233 - val_acc: 0.7864
Epoch 209/500
Epoch 00208: val_acc did not improve
25s - loss: 0.5270 - acc: 0.8716 - val_loss: 1.3018 - val_acc: 0.8409
Epoch 210/500
Epoch 00209: val_acc did not improve
24s - loss: 0.5793 - acc: 0.8511 - val_loss: 1.2839 - val_acc: 0.8545
Epoch 211/500
Epoch 00210: val_acc did not imp

In [51]:
y_pred=model.predict(X_train[1100:])


In [57]:
#tensorboard logdir=. --port 6012


In [53]:
# TURN SIGMOID VALUES INTO BINARY LABEL 
yp=y_pred
for i in range(1100,1351):
    if np.abs(y_pred[i-1100]-1)>0.5:
        yp[i-1100]= 0 
    else:
        yp[i-1100]=1

In [54]:

from sklearn.metrics import confusion_matrix
confusion_matrix(y_train[1100:1351], yp)

array([[163,  19],
       [  7,  62]], dtype=int64)

In [60]:
from sklearn.metrics import classification_report

print(classification_report(y_train[1100:1351], yp))

             precision    recall  f1-score   support

          0       0.96      0.90      0.93       182
          1       0.77      0.90      0.83        69

avg / total       0.91      0.90      0.90       251



In [49]:
model.load_weights('weights-improvement-08-0.90.hdf5')

In [50]:
yp=y_pred
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train[1100:1351], yp)

array([[166,  16],
       [ 49,  20]], dtype=int64)

In [ ]:
metrics.classification_repport(y_train[1100:1351], yp)

In [ ]:
model.load_weights('weights-improvement-24-0.77.hdf5')

In [ ]:
yp=y_pred
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train[1100:1351], yp)

In [67]:
(1351-353)/353

2.8271954674220963

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import *
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
datagen.fit(X_train)
tense = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=1, write_graph=True, write_images=False)
tense.set_model(model)
class_weight = {0 : 1.,1: 3.82}
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max') 
model.fit_generator(datagen.flow(X_train[0:900], y_train[0:900], batch_size=6),steps_per_epoch=90, class_weight = class_weight, epochs=100, verbose=2, validation_data=(X_train[900:1100], y_train[900:1100]),callbacks=[tense,checkpoint])
